## visualize selected call samples

In [ ]:
import pandas as pd
from obspy import read, UTCDateTime
import matplotlib.pyplot as plt
from datetime import timezone
from datetime import timedelta

# Fin Whales

In [ ]:
df = pd.read_csv("/network/projects/aia/whale_call/LABELS/FW/fw_raw.csv")

In [ ]:
first = "2020-10-29 04:10:30.000"
last = "2020-10-29 04:12:30.000"

In [ ]:
df[(df.time_call_start > first) & (df.time_call_end < last)].head()

In [ ]:
# one_group = df[df.group_id == df.group_id.sample().values[0]]
one_group = df[df.group_id == 2395]

In [ ]:
for component in one_group.component.unique():

    one_component = one_group[one_group.component == component].copy()
    one_component['time_call_start_dt'] = pd.to_datetime(one_component.time_call_start) - timedelta(seconds=0.5)
    one_component['time_call_end_dt'] = pd.to_datetime(one_component.time_call_end) - timedelta(seconds=0.5)
    one_component['datetime_start_tz'] = one_component.time_call_start_dt.apply(lambda x : x.replace(tzinfo=timezone.utc))
    one_component['datetime_end_tz'] = one_component.time_call_end_dt.apply(lambda x : x.replace(tzinfo=timezone.utc))

    st = read(one_component.file_path.values[0])
    
    st.filter(
            "bandpass", freqmin=12, freqmax=32, zerophase=True
        )
    
    T0 = UTCDateTime(one_component.time_call_start.min());
    T1= UTCDateTime(one_component.time_call_start.max());

    st_sliced = st.slice(
        starttime=T0 -20,
        endtime=T1 +20)


    # Create a figure and axes object
    fig, ax = plt.subplots(1,1, figsize=(18, 3))

    # Plot the cosine function
    tr = st_sliced[0]
    ax.plot(tr.times("matplotlib"), tr.data, "b-")

    # Plot whales detections on timeline
    for index,row in one_component.iterrows():
        ax.axvline(
            x=row['datetime_start_tz'],
            color='g',
            label="whale Call")
        
        ax.axvline(
            x=row['datetime_end_tz'],
            color='r',
            label="whale Call")

    ax.set_title('Filtered SAC - Component: {}'.format(component))
    st_sliced.spectrogram();

### A selected Fin Whale Detection

In [ ]:
from obspy import read, UTCDateTime

data_path='/network/projects/aia/whale_call/RAW/'
station_name = 'ICQ'
channel = 'HHN'
call_t1 = UTCDateTime("20201029051032")
det_win = 120
data_year = call_t1.year
data_month = call_t1.month
data_day = call_t1.day

date_string = '%s%02d%02d'%(data_year,data_month,data_day)
file_patterns = data_path+date_string+'/'+ '%s.%02d.%02d.CN'%(data_year,data_month,data_day)+'*'+station_name+'*Z*.SAC'

st = read(data_path+date_string+"/"+'%s.%02d.%02d.CN'%(data_year,data_month,data_day)+".%s..%s.SAC"%(station_name,channel))
# st.taper(max_percentage=0.,type='cosine')
st.filter(
        "bandpass", freqmin=12, freqmax=32, zerophase=True
    )


T0 = UTCDateTime(date_string);
print(date_string)

T1= st[0].meta['starttime'];
rel_t=2*(T0-T1);
call_t1 =call_t1-rel_t;
call_t2 = call_t1+det_win;
print(call_t1)
print(call_t2)

st_sliced = st.slice(starttime=call_t1,endtime=call_t2)
st_sliced.plot()
st_sliced.spectrogram();



## Visualize random samples from raw data

### FW Calls

In [ ]:
fw_path = '/network/projects/aia/whale_call/LABELS/FW/'

fw_filt = pd.read_csv(fw_path+'fw_filt.csv')

### Plot R value histogram

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,4))

fw_filt.R.hist(bins=100,ax=ax[0])
ax[0].set_title("R-index")
fw_filt.SNR.hist(bins=100,ax=ax[1])
ax[1].set_title("SNR")
fig.tight_layout();

#### Plot X calls

In [ ]:
NUMBER_OF_CALLS = 5

In [ ]:
import pandas as pd
import numpy as np
from obspy import read, UTCDateTime
import matplotlib.pyplot as plt
from datetime import timedelta

R0 = 5 # set up the minimum R-index value for stronger signal
SNR0 = 10 # set up the minimum SNR value for stronger signal

In [ ]:
for index,row in fw_filt[(fw_filt['R']>R0) & (fw_filt.SNR > SNR0)].sample(n=5).iterrows():
    print(" ============================================== {} ==============================================".format(index))

    sac_file = row['file_path']
    t0 = UTCDateTime(row['time_window_start'])
    t1 = UTCDateTime(row['time_window_end'])

    call_end_modified = UTCDateTime(pd.to_datetime(row['time_call_end']) - timedelta(seconds=0.5))
    call_end_original = UTCDateTime(pd.to_datetime(row['time_call_end']))

    call_start_modified = UTCDateTime(pd.to_datetime(row['time_call_start']) - timedelta(seconds=0.5))
    call_start_original = UTCDateTime(pd.to_datetime(row['time_call_start']))

    st = read(sac_file)

    st_sliced = st.slice(starttime=t0,endtime=t1)
    data_len = len(st_sliced[0].data)
    fig,ax = plt.subplots(figsize=(12,2))
    ax.plot(np.arange(0,data_len)*0.01,st_sliced[0].data)
    ax.axvline(x=call_start_modified-t0,color='red',label='Start modified')
    ax.axvline(x=call_start_original-t0,color='green',label='Start original')
    ax.axvline(x=call_end_modified-t0,color='red',label='End modified',linestyle='--')
    ax.legend()
    plt.plot();

    st_sliced.spectrogram();